## Preparation
Import the modules and remove previous database (if any).

In [1]:
import sqlite3
import pandas as pd
import os
os.chdir('D:\dataverse_files')
os.getcwd()

'D:\\dataverse_files'

In [2]:
try:
    os.remove('airline2.db')
except OSError:
    pass

## Create database
Create database in the working directory and connect it to SQLite

In [3]:
conn = sqlite3.connect('airline2.db')

## Create tables
Create the tables for airports, carrier and plane-data dataset

In [4]:
planes = pd.read_csv("plane-data.csv")
airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")

planes.to_sql("planes", con = conn, index = False)
airports.to_sql("airports", con = conn, index = False)
carriers.to_sql("carriers", con = conn, index = False)

In [5]:
airports

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


Then, create the table for ontime (several csv files)

In [6]:
c = conn.cursor()

In [7]:
for year in range(2000,2006):
    ontime = pd.read_csv(str(year)+".csv", encoding = "ISO-8859-1")
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)
    
conn.commit()

C:\Users\Thunderobot\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Run queries
Query 1: Find model that has the lowest associated average departure delay.

In [8]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

print(c.fetchone()[0], "has the lowest associated average departure delay.")

737-2Y5 has the lowest associated average departure delay.


Query 2: Find the city that has the highest number of inbound flights (excluding cancelled flights).

In [9]:
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON airports.iata = ontime.Dest
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

print(c.fetchone()[0], "has the highest number of inbound flights (excluding cancelled flights).")

Chicago has the highest number of inbound flights (excluding cancelled flights).


Query 3: Find the carrier that has the highest number of cancelled flights.

In [10]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1 
AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC
''')

print(c.fetchone()[0], "has the highest number of cancelled flights.")

Delta Air Lines Inc. has the highest number of cancelled flights.


Query 4: Find the carrier that has the highest number of cancelled flights, relative to their number of total flights.

In [11]:
c.execute('''
SELECT q1.carrier as carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description as carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')

print(c.fetchone()[0], "has the highest number of cancelled flights, relative to their number of total flights.")

United Air Lines Inc. has the highest number of cancelled flights, relative to their number of total flights.


## Close connection

In [ ]:
conn.close()